In [1]:
import pandas as pd

In [ ]:
# Read dataset
df = pd.read_csv('../../data/raw/combined_dataset.csv')


In [ ]:
# Select only accelerometer and gyrometer
new_df = df[df['sensor'] != 'ori']

df['subject'].nunique()

In [ ]:
# Split into Acceleration and Gyroscope data
acc_df = df[df['sensor'] == 'acc'].drop(columns=['sensor'])
gyro_df = df[df['sensor'] == 'gyro'].drop(columns=['sensor'])

In [ ]:
# Ensure the timestamp is in datetime format
acc_df['timestamp'] = pd.to_datetime(acc_df['timestamp'], unit = 'ns')

gyro_df['timestamp'] = pd.to_datetime(gyro_df['timestamp'],  unit = 'ns')

In [ ]:
# Index the timestamp
acc_df.index = acc_df['timestamp'] 
gyro_df.index = gyro_df['timestamp'] 

In [ ]:
# Delete the timestamp columns
del acc_df['timestamp']
del gyro_df['timestamp']

In [ ]:
# Merge: Keep all gyro timestamps, add acc data
df_merged = gyro_df.merge(acc_df.iloc[:,:3], how="left", left_index=True, right_index=True)
df_merged.interpolate(method="linear", inplace=True)

Here we have a lot of missing accelerometer data beccause gyrometer is 
measured at a higher rate than accelerometer and the chances of it measuring 
at the same exact ms is low. We pick the gyro data to sync accelerometer because it measures with the highest frequency. The empty accelerometer data will be linearly interpolated

In [ ]:
df_merged = df_merged.dropna() # Drop NA

In [ ]:
# Rename columns
df_merged.columns = ['gyr_x', 'gyr_y', 'gyr_z', 'subject', 'category', 'activity', 'trial', 'acc_x', 'acc_y', 'acc_z']

# 2.3 million row remain after llinear interppolation


In [ ]:
# Creating the subject DataFrame from Mobifall description
subject_data = {
    "id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 29, 30, 31],
    "age": [32, 26, 26, 32, 36, 22, 25, 22, 30, 26, 26, 29, 24, 24, 25, 27, 25, 25, 25, 26, 25, 27, 47, 27],
    "height": [180, 169, 164, 186, 160, 172, 189, 183, 177, 170, 168, 178, 177, 178, 173, 172, 173, 176, 161, 178, 180, 186, 172, 170],
    "weight": [85, 64, 55, 93, 50, 62, 80, 93, 102, 90, 80, 83, 62, 85, 82, 56, 67, 73, 63, 71, 70, 103, 90, 75],
    "gender": ["M", "M", "F", "M", "F", "F", "M", "M", "M", "F", "F", "M", "M", "M", "M", "F", "M", "M", "F", "M", "M", "M", "M", "M"]
}

df = pd.DataFrame(subject_data)

# Modifying the id column to add "sub_" prefix
df["id"] = "sub" + df["id"].astype(str)

df.rename(columns={'id': 'subject'}, inplace = True)

In [ ]:
# Add subjects detail
df_merged = df_merged.merge(df, on = 'subject')


In [ ]:
# Select only relevant columns
mobifall_df = df_merged[['acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z','age','height','weight','gender', 'category']]


In [ ]:
# Export to csv
mobifall_df.to_csv('../../data/interim/mobifall_df.csv', index = False)